# Setup

In [ ]:
#Check directory
#!pwd

In [ ]:
#Change directory
#%cd /Users/georgesmith/Desktop/RTP/DONOR DATA ANALYSIS

In [ ]:
#Import necessary packages:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import datetime
from datetime import date
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import display
import sys
#Install pingouin package into this jupyter kernal
#!{sys.executable} -m pip install pingouin
import pingouin as pg #this throws the warning message
#!{sys.executable} -m pip install --upgrade statsmodels
import pickle
import plotly.offline as py
color = sns.color_palette()
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import plotly.tools as tls
#import sys
#!{sys.executable} -m pip install pandas_profiling
import pandas_profiling

%matplotlib inline

In [ ]:
#Read in donation data:
donations = pd.read_csv('/Users/georgesmith/Desktop/RTP/DONOR DATA ANALYSIS/donations_by_acct_contact_July_2014_June_29_2020.csv')

#Review import:
donations.head()

In [ ]:
donations.shape

In [ ]:
#Get variable dataypes and non-null counts:
donations.info()

In [ ]:
def col_dtype_conversions(df, col_list1, new_dtype1, col_list2, new_dtype2):
    """
    Converts columns in col_list1 to datatype specified in new_dtype1
    and converts those in col_list2 to datatype specified in new_dtype2

    Params
    ------
    df : pandas.DataFrame
        dataframe with columns in both col_lists
    col_list1 : list
        list of column names to be converted to dtype specified by new_dtype1
    new_dtype1 : str (must put in quotes)
        data type to which columns in col_list1 will be converted
    col_list2 : list
        list of column names to be converted to dtype specified by new_dtype2
    new_dtype2 : str (must put in quotes)
        data type to which columns in col_list2 will be converted

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns, some of which are now cast as
        the specified data types
    """
    for col1 in col_list1:
        df[col1] = df[col1].astype(new_dtype1)
        for col2 in col_list2:
            df[col2] = df[col2].astype(new_dtype2)
    return

In [ ]:
col1 = ['fiscal_year', 'transaction_id']
col2 = ['close_date', 'first_donation_date', 'last_donation_date', 'account_created_date']

In [ ]:
col_dtype_conversions(donations, col1, 'str', col2, 'datetime64[ns]')

In [ ]:
#Confirm dtype conversions
donations.dtypes

In [ ]:
#Remove all refunded donation lineitems to omit them from analysis and calculations
donations = donations[donations['stage'] != 'Refunded']

In [ ]:
donations.shape

In [ ]:
#Show descriptive statistics for non-numeric columns in donations df
#Top - most common value in given column; freq - frequency of column's most common value
donations.describe(include=['O']).T

In [ ]:
#Show descriptive statistics for numeric columns in donations df
#Top - most common value in given column; freq - frequency of column's most common value
donations.describe().T

In [ ]:
#Remove columns that are already clearly extraneous and/or lacking variability
donations.drop(['matching_close_date'], axis=1, inplace=True)

# Initial research question: are donations (volume and dollar total) increasing overtime?

In [ ]:
import pandas_profiling

In [ ]:
# For a quick overview of all the features, levels etc, along with the numeric correlations, 
# https://github.com/pandas-profiling/pandas-profiling
profile = pandas_profiling.ProfileReport(donations)
profile.to_file("donations_profile.html") # download it for viewing in the browser
profile # ATTEMPT to render it in kernal

Address nulls.

Fund Column:

In [ ]:
#Convert nan in fund column (except for pledges) to Annual Fund.
#Also, replace other AF categories with 'Annual Fund'
donations.loc[(((donations['fund'].isna()) | (donations['fund'].isin(["Annual Fund Historical", 'ANN']))) & (donations['donation_record_type'] != 'Pledge')), 'fund'] = "Annual Fund"

In [ ]:
donations.loc[(donations['donation_record_type'] != 'Pledge') & (donations['account_record_type'] != 'Business') & (donations['fund'].isna()), 'fund'] = 'Annual Fund'

In [ ]:
#Group levels referencing the same fund under common levels
donations.loc[(donations['fund'].isin(["25th Anniversary Campaign Fund", '25th Anniversary Fund', '25TH'])), 'fund'] = "25th Anniversary Capital Campaign"
donations.loc[(donations['fund'] == 'NEXT'), 'fund'] = "Next Stage Campaign"
donations.loc[(donations['fund'] == 'John Knapp -Tim Gillham Fund'), 'fund'] = "New Works (Knapp-Gillham) Fund"
donations.loc[(donations['fund'].isin(["Education Program Fund", 'EDUC'])), 'fund'] = "Education Fund"

In [ ]:
#Check fund column value changes
donations.fund.value_counts()

In [ ]:
donations.payment_type.value_counts()

In [ ]:
#Group levels referencing the same payment_type under common levels
donations.loc[(donations['payment_type'] == 'CRED'), 'payment_type'] = "Credit Card"
donations.loc[(donations['payment_type'] == 'Third Party'), 'payment_type'] = "Credit Card - Third Party"
donations.loc[(donations['payment_type'] == 'INKND'), 'payment_type'] = "In Kind"

In [ ]:
donations.payment_type.value_counts()

In [ ]:
donations.donation_record_type.value_counts(dropna=False)

In [ ]:
#Define function to show value counts of categorical vars specified in column list input
def cat_col_value_counts(df, cat_col_list):
    x=[]
    for cat_col in cat_col_list:
         x.append(df[cat_col].value_counts(dropna= False))
    print (x)

In [ ]:
cat_col_list = ['account_record_type', 'type', 'frequency', 'payment_schedule', 'stage', 'card_type', 'donation_origin', 'mailing_city', 'mailing_state']

In [ ]:
cat_col_value_counts(donations, cat_col_list)

Take a look a donations marked with stage == 'Pledged' as well as those marked with type == 'Pledge Payment'

In [ ]:
donations.loc[(donations['stage'] == 'Pledged'), ['account_name','stage', 'frequency', 'type', 'payment_schedule', 'donation_origin', 'amount', 'payment_type', 'pledge_amount', 'amount_paid', 'close_date']]

In [ ]:
donations.loc[(donations['type'] == 'Pledge Payment'), ['account_name','stage', 'frequency', 'type', 'payment_schedule', 'donation_origin', 'amount', 'payment_type', 'pledge_amount', 'amount_paid', 'close_date']]

In [ ]:
donations.loc[(donations['close_date'] > '2015-11-01') & (donations['close_date'] < '2016-01-31'), ['amount', 'close_date', 'donation_record_type', 'payment_type']]

Create campaign column

In [ ]:
donations.loc[(donations['close_date'] > '2020-06-05') & (donations['close_date'] < '2020-08-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Spring 2020 Annual Fund Appeal (S27)'
donations.loc[(donations['close_date'] > '2019-11-19') & (donations['close_date'] < '2020-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2019 Annual Fund Appeal (S27)'
donations.loc[(donations['close_date'] > '2019-05-20') & (donations['close_date'] < '2019-08-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Spring 2019 Annual Fund Appeal (S26)'
donations.loc[(donations['close_date'] > '2018-11-25') & (donations['close_date'] < '2019-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2018 Annual Fund Appeal (S26)'
donations.loc[(donations['close_date'] > '2018-05-20') & (donations['close_date'] < '2018-08-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Spring 2018 Annual Fund Appeal (S25)'
donations.loc[(donations['close_date'] > '2017-11-10') & (donations['close_date'] < '2018-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2017 Annual Fund Appeal (S25)'
donations.loc[(donations['close_date'] > '2016-11-14') & (donations['close_date'] < '2017-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2016 Annual Fund Appeal (S24)'
donations.loc[(donations['close_date'] > '2015-11-14') & (donations['close_date'] < '2016-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2015 Annual Fund Appeal (S23)'
donations.loc[(donations['close_date'] > '2014-11-14') & (donations['close_date'] < '2015-02-01') & (donations['fund'] == 'Annual Fund'), 'campaign'] = 'Fall 2014 Annual Fund Appeal (S22)'
donations.loc[(donations['type'] == 'GIVINGTUESDAYNOW'), 'campaign'] = 'GIVINGTUESDAYNOW (S27)'

In [ ]:
donations.loc[(donations['close_date'] > '2019-06-30') & (donations['close_date'] < '2020-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S27)'
donations.loc[(donations['close_date'] > '2018-06-30') & (donations['close_date'] < '2019-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S26)'
donations.loc[(donations['close_date'] > '2017-06-30') & (donations['close_date'] < '2018-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S25)'
donations.loc[(donations['close_date'] > '2016-06-30') & (donations['close_date'] < '2017-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S24)'
donations.loc[(donations['close_date'] > '2015-06-30') & (donations['close_date'] < '2016-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S23)'
donations.loc[(donations['close_date'] > '2014-06-30') & (donations['close_date'] < '2015-07-01') & (donations['campaign'].isna()), 'campaign'] = 'Outside of Campaign Period (S22)'

In [ ]:
donations.campaign.value_counts()

In [ ]:
#Visualize donations over time
dons = donations.plot.scatter('close_date', 'amount')

In [ ]:
donations.loc[donations['amount']>40000, ['donation_name', 'amount', 'close_date', 'fund']]

In [ ]:
#Remove the 6 outliers above (those above 40k) as well as donations marked stage == 'Refunded'
less_max = donations[(donations['amount']<40000) & (donations['donation_record_type'] != 'Pledge')]

In [ ]:
less_max_scatter = less_max.plot.scatter('close_date', 'amount')

In [ ]:
#Find donation volume by date (excludes donations over 40,000 and refunds)
donations_by_date = less_max.groupby('close_date')['donation_id'].count()
donations_by_date.head()

In [ ]:
#Plot the daily donation volume

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=donations_by_date.index, y=donations_by_date.values, color='purple')

#Assign axis labels:
plt.ylabel('Number of Donations', fontsize=14)
plt.xlabel('Date', fontsize=14)

In [ ]:
#Which date over the analysis period had the largest donation volume?
donations.close_date.mode()

#04-19-2018 makes sense as it was our first giveoutday

In [ ]:
#Fill nan in donations['amount'] with 0
donations.loc[donations['amount'].isna(), 'amount'] = 0
donations[donations['donation_record_type'] == 'Pledge']

In [ ]:
#Show all donations by fiscal year
donations_by_fiscal_year = donations[(donations['donation_record_type'] != 'Pledge')].groupby('fiscal_year')['amount'].sum()
donations_by_fiscal_year

In [ ]:
#Show donations by fund and fiscal year
donations_by_fy_fund = donations[(donations['donation_record_type'] != 'Pledge')].groupby(['fund','fiscal_year'])['amount'].sum()
donations_by_fy_fund

In [ ]:
donations.fund.value_counts()

In [ ]:
#Plot the daily donation volume

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=donations_by_fiscal_year.index, y=donations_by_fiscal_year.values, color='magenta')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)

In [ ]:
#Visualize annual fund donations by date
af_donations_by_close_date = donations[(donations['fund'] == 'Annual Fund') & (donations['donation_record_type'] != 'Pledge')].groupby('close_date')['amount'].sum()
af_donations_by_close_date.head()

In [ ]:
#Plot the total daily annual fund donations

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=af_donations_by_close_date.index, y=af_donations_by_close_date.values, color='magenta')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Totals Per Day')

In [ ]:
#Visualize total annual fund donations by FY
af_donations_by_fy = donations[(donations['fund'] == 'Annual Fund') & (donations['donation_record_type'] != 'Pledge')].groupby('fiscal_year')['amount'].sum()
af_donations_by_fy

In [ ]:
#Plot the total annual fund donations by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=af_donations_by_fy.index, y=af_donations_by_fy.values, color='magenta')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Totals By Fiscal Year')

In [ ]:
#Visualize mean annual fund donations by FY
af_donations_mean_by_fy = donations[(donations['fund'] == 'Annual Fund') & (donations['donation_record_type'] != 'Pledge')].groupby('fiscal_year')['amount'].mean()
af_donations_mean_by_fy

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=af_donations_mean_by_fy.index, y=af_donations_mean_by_fy.values, color='green')

#Assign axis labels:
plt.ylabel('Mean Donations ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Means By Fiscal Year')

In [ ]:
pd.DataFrame(af_donations_mean_by_fy).T

In [ ]:
#Visualize mean annual fund donations by FY
af_donations_vol_by_fy = donations[(donations['fund'] == 'Annual Fund') & (donations['donation_record_type'] != 'Pledge')].groupby('fiscal_year')['amount'].count()
af_donations_vol_by_fy

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=af_donations_vol_by_fy.index, y=af_donations_vol_by_fy.values, color='purple')

#Assign axis labels:
plt.ylabel('Donation Volume', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Volume By Fiscal Year')

In [ ]:
donations.fund.value_counts()

In [ ]:
donations.campaign.value_counts()

In [ ]:
len(donations.campaign.value_counts())

In [ ]:
donations[(donations['fund'] == 'Annual Fund')].groupby('campaign')['amount'].sum().sort_values()

In [ ]:
donations[(donations['fund'] == 'Annual Fund')].groupby('campaign')['amount'].mean().sort_values()

In [ ]:
#Mean annual fund donations by fiscal year (sorted)
donations[(donations['fund'] == 'Annual Fund')].groupby('fiscal_year')['amount'].mean().sort_values()

In [ ]:
#Visualize donations by campaign
donations_by_campaign = donations[(donations['donation_record_type'] != 'Pledge')].groupby('campaign')['amount'].sum()
donations_by_campaign.reindex(['Outside of Campaign Period (S22)', 'Fall 2014 Annual Fund Appeal (S22)', 'Outside of Campaign Period (S23)', 'Fall 2015 Annual Fund Appeal (S23)', 'Outside of Campaign Period (S24)', 'Fall 2016 Annual Fund Appeal (S24)', 'Outside of Campaign Period (S25)', 'Fall 2017 Annual Fund Appeal (S25)', 'Spring 2018 Annual Fund Appeal (S25)', 'Outside of Campaign Period (S26)', 'Fall 2018 Annual Fund Appeal (S26)', 'Spring 2019 Annual Fund Appeal (S26)', 'Outside of Campaign Period (S27)', 'Fall 2019 Annual Fund Appeal (S27)', 'GIVINGTUESDAYNOW (S27)', 'Spring 2020 Annual Fund Appeal (S27)'])

In [ ]:
#Plot the daily donation volume

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=donations_by_campaign.index, y=donations_by_campaign.values, color='green')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Campaign Period', fontsize=14)
plt.xticks(rotation=90)

In [ ]:
pledges = donations.loc[donations['donation_record_type'] == 'Pledge', 
                        ['donation_id', 'donation_record_type', 'amount',
       'close_date', 'account_name', 'account_id', 'account_record_type',
       'contact_id', 'pledge_amount', 'fund', 
       'fiscal_year', 'type', 'first_donation_date', 'last_donation_amount',
       'last_donation_date', 'lifetime_donation_count',
       'lifetime_donation_total_amount', 'lifetime_order_count',
       'lifetime_single_ticket_purchase_total',
       'lifetime_subscription_purchase_total', 'account_created_date', 'account_phone', 'amount_donated_last_365_days',
       'amount_donated_last_fiscal_year', 'amount_donated_this_fiscal_year',
       'ticket_order_id', 'order_count_last_#_days',
       'mailing_street', 'mailing_city', 'mailing_state', 'mailing_zip',
       'campaign']]

In [ ]:
#Filter refunded out of donations; select columns for analysis
donations = donations.loc[(donations['stage'] != 'Refunded') & (donations['donation_record_type'] != 'Pledge'), 
                          ['donation_id', 'donation_record_type', 'amount',
       'close_date', 'account_name', 'account_id', 'account_record_type',
       'contact_id', 'pledge_amount', 'fund', 
       'fiscal_year', 'type', 'first_donation_date', 'last_donation_amount',
       'last_donation_date', 'lifetime_donation_count',
       'lifetime_donation_total_amount', 'lifetime_order_count',
       'lifetime_single_ticket_purchase_total',
       'lifetime_subscription_purchase_total', 'account_created_date', 'account_phone', 'amount_donated_last_365_days',
       'amount_donated_last_fiscal_year', 'amount_donated_this_fiscal_year',
       'ticket_order_id', 'order_count_last_#_days',
       'mailing_street', 'mailing_city', 'mailing_state', 'mailing_zip',
       'campaign']]

Create stewardship level columns

In [ ]:
#Only Annual Fund dontations made by Household or Individual donors are included in stewardship program totals
donations['stp_total_s27'] = donations.loc[(donations['close_date'] >= '2018-07-01') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s26'] = donations.loc[(donations['close_date'] >= '2017-07-01') & (donations['close_date'] <= '2019-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s25'] = donations.loc[(donations['close_date'] >= '2016-07-01') & (donations['close_date'] <= '2018-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s24'] = donations.loc[(donations['close_date'] >= '2015-07-01') & (donations['close_date'] <= '2017-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')
donations['stp_total_s23'] = donations.loc[(donations['close_date'] >= '2014-07-01') & (donations['close_date'] <= '2016-06-30') & (donations['fund'] == 'Annual Fund') & (donations['account_record_type'].isin(['Household', 'Individual']))].groupby('account_id')['amount'].transform('sum')

In [ ]:
#Fill nan in stp_total_sXX cols with 0 to simplify ufunc (stp_level_generator) below:  
donations[['stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']] = donations.loc[:,['stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']].fillna(0)

In [ ]:
donations.isna().sum()

# Translater stewardship amount by season into stewardship level by account_id

In [ ]:
def stp_level_generator(series):
    if series == 0:
        return 'Did Not Give'
    elif (series > 0) & (series < 125):
        return 'Pre-Steward'
    elif (series >= 125) & (series < 250):
        return 'Supporter'
    elif (series >= 250) & (series < 500):
        return 'Investor'
    elif (series >= 500) & (series < 1000):
        return 'Underwriter'
    elif (series >= 1000) & (series < 2500):
        return 'Performer'
    elif (series >= 2500) & (series < 5000):
        return 'Director'
    elif (series >= 5000):
        return 'Producer'

In [ ]:
#As DICTIONARY:
# stp_amt_list = ['stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']
# donor_levels_by_season = {}
# donor_levels_by_season['account_id'] = donations['account_id']
# for col in stp_amt_list:
#     df_name = col[:3] + col[9:] + 'level'
#     donor_levels_by_season[df_name] = (donations[col].apply(stp_level_generator))

In [ ]:
#Create new dataframes with account_id, stp_total_sX cols and corresponding stp_level cols: 
stp_amt_list = ['stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_total_s23']
levels_by_season_df = pd.DataFrame()
levels_by_season_df['account_id'] = donations['account_id']

for col in stp_amt_list:
    levels_by_season_df[col] = donations[col]
    col_name = col[:3] + col[9:] + '_level'
    levels_by_season_df[col_name] = (donations[col].apply(stp_level_generator))

In [ ]:
levels_by_season_df.tail()

In [ ]:
levels_by_season_df.columns

In [ ]:
donor_levels_by_season.keys()

In [ ]:
for col in stp_level_list:
    print(donations[col].value_counts().sort_values())

In [ ]:
levels_by_season_df.head(50)

# DeDupe the level cols of the dataframe (above) by account_id

In [ ]:
levels_by_season_df

In [ ]:
type(levels_by_season_df.loc[:,['account_id','stp__s27_level']])

In [ ]:
levels_by_season_df.loc[:,['account_id','stp__s27_level']].drop_duplicates(subset='account_id', keep='last')

In [ ]:
(levels_by_season_df['stp__s27_level'].drop_duplicates(subset='account_id', keep='last')).loc[:, ['account_id', 'account_name', 'stp__s27_level']]

In [ ]:
for col in stp_level_list:
    [col + '_']
    (donations.drop_duplicates(subset=['account_id', col], keep='last')).loc[:, ['account_id', 'account_name', col]]

In [ ]:
d = pd.DataFrame()
for i, col in enumerate stp_level_list:
    

In [ ]:
donations.loc[donations['stp_s26level'] == 'Producer'].drop_duplicates(subset='account_id', keep='last')

In [ ]:
donations['test1'] = donations.stp_total_s27.apply(stp_level_generator)
donations['test1'].value_counts()

In [ ]:
donations.columns

In [ ]:
donations.tail(50)

In [ ]:
for i in range(len(donations)):
     if (donations.loc[i, 'stp_total_s27'] > 0) & (donations.loc[i, 'stp_total_s27'] < 125):
        level[s27] = 'Pre-Steward'
    elif (donations.loc[i, 'stp_total_s27'] > 125) & (donations.loc[i, 'stp_total_s27'] < 5000):
        level[s27] = 'Stewards'
    elif donations.loc[i, 'stp_total_s27'] > 5000:
        level[s27] = 'money bags'

In [ ]:
 donations.shape

In [ ]:
for col1 in col_list1:
        df[col1] = df[col1].astype(new_dtype1)
        for col2 in col_list2:
            df[col2] = df[col2].astype(new_dtype2)
    return

# Retention

In [ ]:
#donations[donations['']]

In [ ]:
stp_total_2019.head()

Separate continuous and categorical features for exploration.

In [ ]:
#Assign continuous columns
cont = donations.iloc[:, np.r_[3:5, 9, 17:26, 28, 34, 41:44, 45, 47, 50]]
cont.head()

In [ ]:
#Select remaining variables for cats df (categorical variables)
features = donations.columns
cont_vars = cont.columns
cat_vars = [f for f in features if f not in cont_vars]

In [ ]:
#Create cont df
cats = donations.loc[:, cat_vars]

#Add in the continuous target variable: 'amount'
cats['amount'] = cont['amount']

#Review head records
cats.head()

In [ ]:
cats['donation_record_type'].value_counts().plot.bar(title='Freq dist of Donation Record Type', figsize=[15,10])

In [ ]:
donations['fund'].value_counts().plot.bar(title='Freq dist of Fund', figsize=[15,10])

In [ ]:
cont_vars

Continuous Plots

In [ ]:
x = less_max['amount']
ax = sns.distplot(x)

In [ ]:
fig, ax = plt.subplots(len(cont_vars), figsize=(16,12))

for i, col_val in enumerate(cont_vars):
    try:
        sns.distplot(cont[col_val], hist=True, ax=ax[i])
        ax[i].set_title('Freq dist '+col_val, fontsize=10)
        ax[i].set_xlabel(col_val, fontsize=8)
        ax[i].set_ylabel('Count', fontsize=8)
    except RuntimeError as re:
        if str(re).startswith("Selected KDE bandwidth is 0. Cannot estimate density."):
            sns.distplot(cont[col_val], hist=True, ax=ax[i], kde_kws={'bw': 0.1})
        else:
            raise re
    
plt.show()

In [ ]:
cont_dollars = ['amount', 'pledge_amount', 'amount_matched', 'last_donation_amount','lifetime_donation_total_amount',
       'lifetime_single_ticket_purchase_total', 'lifetime_subscription_purchase_total', 'amount_paid', 'amount_donated_last_365_days',
       'amount_donated_last_fiscal_year', 'amount_donated_this_fiscal_year', 'group_sales_total', 'total_amount']

In [ ]:
fig, ax = plt.subplots(len(cont_dollars), figsize=(16,12))

for i, col_val in enumerate(cont_dollars):
    try:
        sns.distplot(less_max[col_val], hist=True, ax=ax[i])
        ax[i].set_title('Freq dist '+col_val, fontsize=10)
        ax[i].set_xlabel(col_val, fontsize=8)
        ax[i].set_ylabel('Count', fontsize=8)
    except RuntimeError as re:
        if str(re).startswith("Selected KDE bandwidth is 0. Cannot estimate density."):
            sns.distplot(less_max[col_val], hist=True, ax=ax[i], kde_kws={'bw': 0.1})
        else:
            raise re
    
plt.show()

In [ ]:
fig, ax = plt.subplots(len(cont_vars), figsize=(16,12))

for i, col_val in enumerate(cont_vars):
    try:
        sns.distplot(cont[col_val], hist=True, ax=ax[i])
        ax[i].set_title('Freq dist '+col_val, fontsize=10)
        ax[i].set_xlabel(col_val, fontsize=8)
        ax[i].set_ylabel('Count', fontsize=8)
    except RuntimeError as re:
        if str(re).startswith("Selected KDE bandwidth is 0. Cannot estimate density."):
            sns.distplot(cont[col_val], hist=True, ax=ax[i], kde_kws={'bw': 0.1})
        else:
            raise re
    
plt.show()